In [1]:
%cd ./code

/Users/paulp/Documents/Personal/Study/MSc/COMP7607/FinalProject/subnetwork-probing/code


In [2]:
import sys
from visualize import visualize_head_sparsity, visualize_dense_sparsity, visualize_layer_attn_sparsity
from matplotlib import pyplot as plt
plt.switch_backend('agg')
import numpy as np
from classifiers import ModelEncoder
from bert import WordLevelBert
from masked_pretrained_models import MaskedWordLevelModel
from pretrained_models import WordLevelModel
from masked_bert import MaskedWordLevelBert
from train import train_ner, train_ud, train_pos, save_mask_scores
from visualize import visualize_head_sparsity, visualize_dense_sparsity, visualize_layer_attn_sparsity

/opt/miniconda3/envs/subnet_prob/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Not using CUDA!


[nltk_data] Downloading package punkt to /Users/paulp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
mask_lr_base = 0.2
lr_base = 5e-05
lambda_init = 0
lambda_final = 125
batch_size = 256
verbose = True
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [6]:
#choose the UD task, pretrained setting,prune method
task = 'UD'
setting = 'pretrained'
method = 'prune'
params_list = ((768,768), (768,192), (768,24), (768,6), 
                               (768,1), (192,1), (24,1), (6,1), (1,1))
params = params_list[4]
#rank = params
epochs = 20
out_w_per_mask, in_w_per_mask = params
model_name = 'roberta-base'
masked = True
kwargs = {'lambda_init' : lambda_init, 'lambda_final' : lambda_final, 
        'epochs' : epochs, 'lr_base' : lr_base, 'mask_lr_base' : mask_lr_base, 
        'verbose' : verbose, 'masked' : masked, 
        "batch_size": batch_size}


In [8]:
!mkdir ../Results

## 1. Train subnetwork and log accuracy

In [ ]:
#subnetwork 
pretrained_model = MaskedWordLevelModel(model_name, out_w_per_mask, in_w_per_mask, verbose=True)
pretrained_encoder = ModelEncoder(pretrained_model, mlp1 = False)

In [ ]:
# get model size
def get_num_params(model: nn.Module) -> int:
    """get the number of parameters in a model"""
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params
get_num_params(pretrained_model)

log, model = train_ud(pretrained_encoder, 
                    '../UD_English/en-ud-train.conllu', 
                    '../UD_English/en-ud-dev.conllu', 
                    **kwargs)
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
save_mask_scores(model, log, base = path)  

for key in log[0].keys():
    plt.plot(np.arange(len(log)), [ a[key] for a in log], color='blue')
    plt.title(key)
    plt.savefig('{}_{}_graph.png'.format(path, key))
    plt.show(block=False)
    plt.close()



In [ ]:
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
if masked:
     visualize_head_sparsity(model, path, block = False)
     visualize_dense_sparsity(model, path, block = False)
     visualize_layer_attn_sparsity(model, path, block = False)
     print("Percentage of elements within 0.01 of 0 or 1: {:.5f}".format(bert.compute_binary_pct()))

In [ ]:
sparsities = np.array([model.get_sparsity_layer_attn(l).item() for l in range(12)])
sparsities = sparsities / np.sum(sparsities)
sparsities

array([0.27118644, 0.05932203, 0.30084745, 0.05084746, 0.12288136,
       0.11864407, 0.06779661, 0.        , 0.00423729, 0.00423729,
       0.        , 0.        ])

## 2. Train with weights reset

In [ ]:
#subnetwork 
pretrained_model = MaskedWordLevelModel(model_name, out_w_per_mask, in_w_per_mask, verbose=True)
pretrained_encoder = ModelEncoder(pretrained_model, mlp1 = False)
pretrained_model.reset_weights(encoder_only = False)

In [ ]:
log, model = train_ud(pretrained_encoder, 
                    '../UD_English/en-ud-train.conllu', 
                    '../UD_English/en-ud-dev.conllu', 
                    **kwargs)
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
save_mask_scores(model, log, base = path)  

for key in log[0].keys():
    plt.plot(np.arange(len(log)), [a[key] for a in log], color='blue')
    plt.title(key)
    plt.savefig('{}_{}_graph.png'.format(path, key))
    plt.show(block=False)
    plt.close()



In [ ]:
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
if masked:
     visualize_head_sparsity(model, path, block = False)
     visualize_dense_sparsity(model, path, block = False)
     visualize_layer_attn_sparsity(model, path, block = False)
     print("Percentage of elements within 0.01 of 0 or 1: {:.5f}".format(bert.compute_binary_pct()))

## 3. MLP with original weights

In [ ]:
#mlp1
#bert = WordLevelBert(model_name)
#bert.freeze_bert()
#bert_encoder = BertEncoder(bert, mlp1 = True, rank = rank)
#bert.reset_weights(encoder_only = False)
#fine tuning
#bert = WordLevelBert(model_name)
#bert_encoder = BertEncoder(bert, mlp1 = False)

In [9]:
# mpl
pretrained_model = MaskedWordLevelModel(model_name, out_w_per_mask, in_w_per_mask, verbose=True)
pretrained_encoder = ModelEncoder(pretrained_model, mlp1 = True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
log, model = train_ud(pretrained_encoder, 
                    '../UD_English/en-ud-train.conllu', 
                    '../UD_English/en-ud-dev.conllu', 
                    **kwargs)
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
save_mask_scores(model, log, base = path)  

for key in log[0].keys():
    plt.plot(np.arange(len(log)), [a[key] for a in log], color='blue')
    plt.title(key)
    plt.savefig('{}_{}_graph.png'.format(path, key))
    plt.show(block=False)
    plt.close()



Removed length 159
Removed length 135
lambda_init: 0, lambda_final: 125, lambda_startup_frac: 0.25, lambda_warmup_frac: 0.5
lr_base: 5e-05, mask_lr_base: 0.2, lr_warmup_frac: 0.1, epochs: 20, batch_size: 256


/Users/paulp/Documents/Personal/Study/MSc/COMP7607/FinalProject/subnetwork-probing/code/pretrained_models.py:156: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_croot-4qf3nw4h/pytorch_1648016051178/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1356.)
  features_packed = features.masked_select(all_end_mask)
/Users/paulp/Documents/Personal/Study/MSc/COMP7607/FinalProject/subnetwork-probing/code/classifiers.py:140: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_croot-4qf3nw4h/pytorch_1648016051178/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  dep_W_headT.data.masked_fill_(pad_mask.byte(), -float('i

Log: {'dev_acc': 0.004033001097844149, 'loss_val': 71.10604858398438}
Log: {'dev_acc': 0.05493797583159309, 'loss_val': 48.444908142089844}
Log: {'dev_acc': 0.13626078632829386, 'loss_val': 31.11243438720703}
Log: {'dev_acc': 0.14614470046370093, 'loss_val': 18.107484817504883}


In [ ]:
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
if masked:
     visualize_head_sparsity(model, path, block = False)
     visualize_dense_sparsity(model, path, block = False)
     visualize_layer_attn_sparsity(model, path, block = False)
     print("Percentage of elements within 0.01 of 0 or 1: {:.5f}".format(bert.compute_binary_pct()))

## 3. MLP with weights reset

In [ ]:
#mlp1
#bert = WordLevelBert(model_name)
#bert.freeze_bert()
#bert_encoder = BertEncoder(bert, mlp1 = True, rank = rank)
#bert.reset_weights(encoder_only = False)
#fine tuning
#bert = WordLevelBert(model_name)
#bert_encoder = BertEncoder(bert, mlp1 = False)

In [ ]:
# mpl
pretrained_model = MaskedWordLevelModel(model_name, out_w_per_mask, in_w_per_mask, verbose=True)
pretrained_encoder = ModelEncoder(pretrained_model, mlp1 = True)
pretrained_model.reset_weights(encoder_only = False)

In [ ]:
log, model = train_ud(pretrained_encoder, 
                    '../UD_English/en-ud-train.conllu', 
                    '../UD_English/en-ud-dev.conllu', 
                    **kwargs)
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
save_mask_scores(model, log, base = path)  

for key in log[0].keys():
    plt.plot(np.arange(len(log)), [a[key] for a in log], color='blue')
    plt.title(key)
    plt.savefig('{}_{}_graph.png'.format(path, key))
    plt.show(block=False)
    plt.close()



In [ ]:
path = './Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
if masked:
     visualize_head_sparsity(model, path, block = False)
     visualize_dense_sparsity(model, path, block = False)
     visualize_layer_attn_sparsity(model, path, block = False)
     print("Percentage of elements within 0.01 of 0 or 1: {:.5f}".format(bert.compute_binary_pct()))